<a id='Left-Scraping-and-cleaning'></a>

In [1]:
import requests
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
#from langdetect import detect

In [2]:
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000

In [3]:
np.set_printoptions(threshold=np.inf)
pd.options.display.max_colwidth = 1000

- List of the people I considered for this reasearch

In [82]:
lista = ['Noam Chomsky',
         'Yanis Varoufakis',
         'Jeremy Rifkin',
         'Naomi Klein',
         'John Bellamy Foster', 
         'Raj Patel',
         'Michael Parenti',
         'Michael Hardt',
         'Antonio Negri',
         'George Monbiot',
         'John Gray',  
         'Michel Chossudovsky',
         'Perry Anderson', 
         'Alexander Cockburn',
         'Terry Eagleton',
         'David Harvey',
         'Fredric Jameson',
         'Raymond Williams', 
         'E P Thompson', 
         'Eric Hobsbawm', 
         'Tariq Ali',
         'Angela Davis', 
         'Howard Zinn', 
         'Judith Butler',
         'John Holloway', 
         'Michael Moore',
         'Michael Kazin',
         'Richard D Wolff',
         'Steve Ellner',
         'Branko Milanovic',
         'Anthony Barnett',
         'Micheal Otsuka',
         'Jodi Dean',
         'Abby Martin',
         'Paul Mason',

]

- Getting Data from Goodreads website

In [83]:
#List of name in the format for Goodreads website
new_lista = []
for name in lista:
    new_lista.append(name.replace(' ','+'))
    

In [84]:
new_lista

['Noam+Chomsky',
 'Yanis+Varoufakis',
 'Jeremy+Rifkin',
 'Naomi+Klein',
 'John+Bellamy+Foster',
 'Raj+Patel',
 'Michael+Parenti',
 'Michael+Hardt',
 'Antonio+Negri',
 'George+Monbiot',
 'John+Gray',
 'Michel+Chossudovsky',
 'Perry+Anderson',
 'Alexander+Cockburn',
 'Terry+Eagleton',
 'David+Harvey',
 'Fredric+Jameson',
 'Raymond+Williams',
 'E+P+Thompson',
 'Eric+Hobsbawm',
 'Tariq+Ali',
 'Angela+Davis',
 'Howard+Zinn',
 'Judith+Butler',
 'John+Holloway',
 'Michael+Moore',
 'Michael+Kazin',
 'Richard+D+Wolff',
 'Steve+Ellner',
 'Branko+Milanovic',
 'Anthony+Barnett',
 'Micheal+Otsuka',
 'Jodi+Dean',
 'Abby+Martin',
 'Paul+Mason']

In [85]:
#Scraping all the existing quotes for the person I need from Goodreads
quotes = []
author = []

for name in new_lista:
    second_temp_quote = []
    second_temp_author = []
    r = requests.get("https://goodquotesapi.herokuapp.com/author/{}".format(name))
    data = r.json()
    if len(data) < 1:
        pass
    else:
        for page in tqdm_notebook(range(1,data['total_pages']+1)):
            new_r = requests.get("https://goodquotesapi.herokuapp.com/author/{}?page={}".format(name,page))
            new_data = new_r.json()
            first_temp_quote = []
            first_temp_author = []

            for i in range(20):
                temp_quote = []
                temp_author = []

                try: 
                    temp_quote.append(new_data['quotes'][i]['quote'])
                    temp_author.append(new_data['quotes'][0]['author'])

                except:
                    temp_quote.append(np.nan)
                    temp_author.append(np.nan)
                first_temp_author.append(temp_author)
                first_temp_quote.append(temp_quote)
            second_temp_quote.append(first_temp_quote)
            second_temp_author.append(first_temp_author)
    quotes.append(second_temp_quote)
    author.append(second_temp_author)

In [184]:
#Creating the goodreads Dataframe
goodreads = pd.DataFrame({
    'Author': [item for l in author for sublist in l for j in l for item in sublist], 
    'Quotes': [item for l in quotes for sublist in l for j in l for item in sublist],
    'source': 'goodreads'
})

In [185]:
#Leaving every cell in a string
goodreads.Author = goodreads.Author.apply(lambda x : x[0])
goodreads.Quotes = goodreads.Quotes.apply(lambda x : x[0])

In [186]:
#Deleting all the rows where the Author is not in the list
goodreads = goodreads[goodreads['Author'].isin(lista)]

- Getting data from Wikiquote // I took the data in a different Jupyter Notebook

In [231]:
import wikiquotes

[Wiki scraping / cleaning](../WikiQuote/Left.ipynb#Wiki-Scraping-and-cleaning)

In [187]:
#Loading the previous dataframe scraped from Wikiquotes
wikiquotes = pd.read_csv('../WikiQuote/left.csv')

In [188]:
#Checking the author
wikiquotes.Author.value_counts()

Noam Chomsky                       249
John Gray                          163
Michael Moore                       68
Michael Parenti                     68
Terry Eagleton                      53
Howard Zinn                         45
Eric Hobsbawm                       42
Angela Davis                        40
David Harvey                        38
Naomi Klein                         38
Perry Anderson                      32
Michel Chossudovsky                 24
Jeremy Rifkin                       18
The Chomsky Reader                  18
John Holloway (sociologist)         17
Fredric Jameson                     16
Tariq Ali                           15
Yanis Varoufakis                    12
Michael Hardt and Antonio Negri     12
John Bellamy Foster                 11
George Monbiot                       9
Van Jones                            9
Judith Butler                        8
Alexander Cockburn                   7
Mumia Abu-Jamal                      7
Raymond Williams         

In [189]:
wikiquotes = wikiquotes.drop('Political Orientation', axis=1)

In [190]:
wikiquotes['source'] = 'wikiquotes'

In [191]:
import re

In [192]:
wikiquotes.Author = wikiquotes.Author.apply(lambda x : re.sub(r'\(.*?\)', '', x))

- Getting Data from BrainQuote

In [193]:
#Creating a list for the Brainquote format
brain_lista = []
for n in lista:
    brain_lista.append(n.lower().replace(' ','_'))
    

In [194]:
brain_lista

['noam_chomsky',
 'yanis_varoufakis',
 'jeremy_rifkin',
 'naomi_klein',
 'john_bellamy_foster',
 'raj_patel',
 'michael_parenti',
 'michael_hardt',
 'antonio_negri',
 'george_monbiot',
 'john_gray',
 'michel_chossudovsky',
 'perry_anderson',
 'alexander_cockburn',
 'terry_eagleton',
 'david_harvey',
 'fredric_jameson',
 'raymond_williams',
 'e_p_thompson',
 'eric_hobsbawm',
 'tariq_ali',
 'angela_davis',
 'howard_zinn',
 'judith_butler',
 'john_holloway',
 'michael_moore',
 'michael_kazin',
 'richard_d_wolff',
 'steve_ellner',
 'branko_milanovic',
 'anthony_barnett',
 'micheal_otsuka',
 'jodi_dean',
 'abby_martin',
 'paul_mason']

In [195]:
#Scraping the quotes from Brainquote website
q = []
a = []

for name in brain_lista:
    temp_quote = []
    temp_author = []
    r = requests.get("https://www.brainyquote.com/authors/{}".format(name))
    soup = BeautifulSoup(r.text,'html.parser')
    for each in soup.find_all('a', attrs={'title':"view quote"}):
        temp_quote.append(each.text)
        #if len(temp_quote) > 0:
        temp_author.append(name)
    q.append(temp_quote)
    a.append(temp_author)
    


In [196]:
#Creating the Brainquote dataframe
brain = pd.DataFrame({'Author': [item for l in a for item in l],
                      'Quotes': [item for l in q for item in l],
                      'source': 'brainquote'})

In [197]:
#Removing '_' from the name and capitalize Name and Surname
brain.Author = brain.Author.apply(lambda x : x.replace('_',' ').title())

In [216]:
Left_quotes = pd.concat([goodreads, wikiquotes, brain])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [218]:
#Removing Duplicates
Left_quotes = prova.drop_duplicates(subset='Quotes')

In [272]:
#Writing the same string for the same Author
Left_quotes.loc[Left_quotes['Author'].str.contains('Thompson'),'Author']='E. P. Thompson'

In [240]:
#Checking missing values
Left_quotes = prova.dropna()

- Checking the language of the quotes scraped

In [241]:
Left_quotes['language'] = Left_quotes.Quotes.apply(lambda x : detect(x))

In [222]:
Left_quotes.language.value_counts()

en    3891
es      69
ar      29
fr      14
pt      11
tr      10
it       8
ru       8
hr       7
bg       4
id       3
el       3
da       2
de       2
ro       1
so       1
et       1
sw       1
ko       1
vi       1
tl       1
ca       1
Name: language, dtype: int64

In [236]:
Left_quotes = Left_quotes[Left_quotes.language == 'en']

In [237]:
Left_quotes.shape

(3891, 4)

- The Biggest problem: the quotes in the quotes

In [242]:
#Checking for the exact string
dup = {}
#count = 0
for name in Left_quotes.Author.unique():
    conta = []
    for text in Left_quotes[Left_quotes.Author==name].Quotes :
        count = 0
        for i in range(len(Left_quotes[Left_quotes.Author==name])):
            if text in Left_quotes[Left_quotes.Author==name].iloc[i,1]:
                count += 1


        conta.append(count)
    dup[name] = conta

In [243]:
Left_quotes['Counter'] = 0
for name in Left_quotes.Author.unique():
    print(name, ' done')
    Left_quotes.loc[Left_quotes.Author==name, 'Counter'] = dup[name]


Noam Chomsky  done
Yanis Varoufakis  done
Jeremy Rifkin  done
Naomi Klein  done
Antonio Negri  done
Michael Hardt  done
George Monbiot  done
John Gray  done
Terry Eagleton  done
David Harvey  done
Fredric Jameson  done
Raymond Williams  done
Eric Hobsbawm  done
Tariq Ali  done
Howard Zinn  done
Judith Butler  done
Michael Moore  done
The Chomsky Reader  done
John Bellamy Foster  done
Raj Patel  done
Michael Parenti  done
Michael Hardt and Antonio Negri  done
Michel Chossudovsky  done
Perry Anderson  done
Van Jones  done
Alexander Cockburn  done
E. P. Thompson  done
Angela Davis  done
Mumia Abu-Jamal  done
John Holloway   done
E P Thompson  done


In [245]:
Left_quotes.Counter.value_counts()

1      3903
2       131
3        22
4         8
49        1
13        1
5         1
254       1
10        1
Name: Counter, dtype: int64

In [247]:
Left_quotes = Left_quotes[Left_quotes.Counter<2]

In [275]:
Left_quotes.drop(['source', 'language', 'Counter'], axis=1, inplace=True)

In [276]:
#Label for this Dataframe
Left_quotes['Political Orientation'] = 'Left'

In [280]:
#Dropping some Authors
fin_drop = (Left_quotes['Author'] != 'Thomas Piketty') &(Left_quotes['Author'] != 'Michael Hardt and Antonio Negri') & (Left_quotes['Author'] != 'The Chomsky Reader') 

In [281]:
Left_quotes = Left_quotes[fin_drop]

- Saving

In [283]:
Left_quotes.to_csv('left_quotes.csv')

- Part 2

In [303]:
Left_quotes = pd.read_csv('left_quotes.csv')

In [308]:
#Selecting 4 Authors for a total of bit more than 100 quotes for the final test df
list_test = ['Perry Anderson', 'John Holloway ', 'Michel Chossudovsky', 'Raymond Williams']

In [309]:
#Creating the Dataframe for the test
L_test = Left_quotes[Left_quotes['Author'].isin(list_test)]


In [310]:
L_test.Author.value_counts()


Perry Anderson         32
Raymond Williams       29
Michel Chossudovsky    24
John Holloway          17
Name: Author, dtype: int64

In [311]:
#removing the 4 authors from the main database
Left_quotes = Left_quotes[~Left_quotes['Author'].isin(list_test)]

In [313]:
#Creating the "sample Dataframe" for the train
L_train = Left_quotes.copy()

In [314]:
L_train.shape

(3617, 5)

- Saving the 2 dataset

In [315]:
#csv file for the train
L_train.to_csv('train_left.csv', index=False)

In [316]:
#csv file for the test
L_test.to_csv('test_left.csv', index=False)

**SECOND PART**

- Loading the previous saved file to carry on with the cleaning

In [182]:
L_train = pd.read_csv('train_left.csv')

In [183]:
L_test = pd.read_csv('test_left.csv')

In [184]:
#Dropping manually
L_train.drop(L_train.index[[1486,1487, 1917, 1930, 334, 2748, 1898, 963,
                                  353, 2139, 2340, 2359, 2332, 9, 434, 1846, 1992,
                                 1448, 866, 1448, 2530, 1938, 2030, 664, 2256, 1049, 530,
                                 1273, 3270, 2407, 475, 2504, 2503, 2523, 145, 97, 2690,
                                 143,191, 202, 282, 294, 397, 401, 416, 496, 1816, 624,
                                 878, 958, 1900, 1858, 1892, 2204,1878, 1884, 1923, 105, 
                                 364, 2505, 14, 2501, 910, 1149, ]], inplace=True)


- For Noam Chomsky I am going to use only wikiquotes and I will drop the others

In [185]:
L_train=L_train[L_train.Author!='Noam Chomsky']

In [186]:
wiki=pd.read_csv('../WikiQuote/left.csv')

In [187]:
wiki = wiki[wiki.Author=='Noam Chomsky']

In [188]:
L_train = pd.concat([L_train, wiki])

/Users/enrico/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


- Checking for quotes about the Author and not from the author

In [189]:
def find_autoref(df):
    autoref = {}
    for name in df.Author.unique():
        conta = []

        for text in df[df.Author==name].Quotes :
            count = 0

            if name in text or name.split(' ')[-1] in text:
                count += 1

            conta.append(count)
        autoref[name] = conta 
    return autoref

In [190]:
autoref = find_autoref(L_train)

In [191]:
L_train['auto'] = 0
for name in L_train.Author.unique():
    print(name, ' done')
    L_train.loc[L_train.Author==name, 'auto'] = autoref[name]

Yanis Varoufakis  done
Jeremy Rifkin  done
Naomi Klein  done
Antonio Negri  done
Michael Hardt  done
George Monbiot  done
John Gray  done
Terry Eagleton  done
David Harvey  done
Fredric Jameson  done
Eric Hobsbawm  done
Tariq Ali  done
Howard Zinn  done
Judith Butler  done
Michael Moore  done
John Bellamy Foster  done
Raj Patel  done
Michael Parenti  done
Van Jones  done
Alexander Cockburn  done
E. P. Thompson  done
Angela Davis  done
Mumia Abu-Jamal  done
Noam Chomsky  done


In [192]:
L_train=L_train[L_train.auto==0]

- Checking for duplicates after preprocessing the quotes

In [193]:
from textacy.preprocess import preprocess_text
import re


In [204]:
L_train['texicity'] = L_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True
                                                                    ))

In [ ]:
stops = stopwords.words

In [209]:
def tok_stop(df):
    df['texicity'] = df['texicity'].apply(lambda x: x.split())
    df['texicity'] = df['texicity'].apply(lambda x : [item for item in x if item not in stops])
    df['texicity'] = df['texicity'].apply(lambda x: ' '.join(x))
    df['texicity'] = df['texicity'].apply(lambda x: re.sub(r'\d+', '',x)) 
    return df
    

In [210]:
L_train = tok_stop(L_train)

- Looking for other duplicates

In [212]:
fake = {}
#count = 0
for name in L_train.Author.unique():
    print(name)
    conta = []
    for j,text in enumerate(L_train[L_train.Author==name].texicity):
        print(j)
        count = 0
        for i in range(len(L_train[L_train.Author==name])):
            if text in L_train[L_train.Author==name].iloc[i,6]:
                count += 1


        conta.append(count)
    fake[name] = conta

Yanis Varoufakis
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
Jeremy Rifkin
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Naomi Klein
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63

79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
John Bellamy Foster
0
1
2
3
4
5
6
7
8
9
10
Raj Patel
0
1
2
3
4
5
6
Michael Parenti
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
Van Jones
0
1
2
3
4
5
6
7
8
Alexander Cockburn
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
E. P. Thompson
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Angela Davis
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
Mumia Abu-Jamal
0
1
2
3
4
5
6
Noam Chomsky
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66

In [214]:
for name in L_train.Author.unique():
    print(name)
    L_train.loc[L_train.Author==name, 'rep'] = fake[name]


Yanis Varoufakis
Jeremy Rifkin
Naomi Klein
Antonio Negri
Michael Hardt
George Monbiot
John Gray
Terry Eagleton
David Harvey
Fredric Jameson
Eric Hobsbawm
Tariq Ali
Howard Zinn
Judith Butler
Michael Moore
John Bellamy Foster
Raj Patel
Michael Parenti
Van Jones
Alexander Cockburn
E. P. Thompson
Angela Davis
Mumia Abu-Jamal
Noam Chomsky


In [215]:
L_train.rep.value_counts()

1.0    2734
2.0      67
4.0       1
Name: rep, dtype: int64

In [216]:
L_train=L_train[L_train.rep==1]

- saving

In [218]:
L_train.to_csv('L_train_clean.csv', index=False)

- Same for the test dataset

In [220]:
autoref_test = find_autoref(L_test)

In [221]:
L_test['auto'] = 0
for name in L_test.Author.unique():
    print(name, ' done')
    L_test.loc[L_test.Author==name, 'auto'] = autoref_test[name]

Raymond Williams  done
Michel Chossudovsky  done
Perry Anderson  done
John Holloway  done


In [222]:
L_test=L_test[L_test.auto==0]

In [223]:
L_test['texicity'] = L_test.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True))

In [224]:
L_test=tok_stop(L_test)

In [226]:
fake = {}
#count = 0
for name in L_test.Author.unique():
    print(name)
    conta = []
    for i,text in enumerate(L_test[L_test.Author==name].texicity) :
        print(i)
        count = 0
        for i in range(len(L_test[L_test.Author==name])):
            if text in L_test[L_test.Author==name].iloc[i,6]:
                count += 1


        conta.append(count)
    fake[name] = conta

Raymond Williams
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Michel Chossudovsky
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
Perry Anderson
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
John Holloway
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [227]:
L_test['rep'] = 0
for name in L_test.Author.unique():
    print(name, ' done')
    L_test.loc[L_test.Author==name, 'rep'] = fake[name]


Raymond Williams  done
Michel Chossudovsky  done
Perry Anderson  done
John Holloway  done


In [228]:
L_test.rep.value_counts()

1    93
2     2
Name: rep, dtype: int64

In [229]:
L_test=L_test[L_test.rep==1]

In [230]:
L_test.to_csv('L_test_clean.csv', index=False)

**Part 3**

- Duplicate by n_gram

In [458]:
L_train = pd.read_csv('L_train_clean.csv')

In [459]:
L_train['Political Orientation']='Left'

In [460]:
L_train['texicity'] = L_train.Quotes.apply(lambda x : preprocess_text(x, 
                                                                      lowercase=True, 
                                                                      no_punct=True, 
                                                                      no_accents=True, 
                                                                      no_contractions=True))

In [461]:
L_train=tok_stop(L_train)

In [463]:
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

In [464]:
def bag_word(df, orientation, mm, mx, col):
    for i in range(mm,mx):
        vect = CountVectorizer(stop_words='english', ngram_range=(i,i))
        words = {}
        lefty = df[df['Political Orientation']==orientation]
        Left_Voc = ''.join(lefty[col])
        lefty_n_grams = vect.build_analyzer()(Left_Voc)
        words['Left'] = Counter(lefty_n_grams).most_common()
        print('Number of gram: ', i)
        print(Counter(lefty_n_grams).most_common(100))
        print()
        print()
        return Counter(lefty_n_grams).most_common()

In [1]:
L_train.loc[L_train['texicity'].str.contains('million'), 'texicity'] 

In [2]:
list_gram = bag_word(L_train, 'Left', 8,9,'texicity' )

In [466]:
list_ind=[]
for gram in list_gram:
    if gram[1]>1:
        try:
            (one,two)=(L_train.loc[L_train['texicity'].str.contains(gram[0]), 'texicity'].index)
            if len(L_train.loc[one,'texicity'])>len(L_train.loc[two,'texicity']):
                list_ind.append(two)
            else:
                list_ind.append(one)

        except:
            pass

In [467]:
l=list(set(list_ind))

In [469]:
L_train=L_train.drop(l)


In [480]:
#Dropping manually
l_2 = [308, 286, 127, 1488, 1397, 1552, 2485, 1410]

In [481]:
L_train=L_train.drop(l_2)

- saving the final csv

In [491]:
L_train.to_csv('L_train_clean.csv')